# TrainScannerNetwork.ipynb

## Author: Daniel Mallia
## Date Begun: 1/17/2020

**This Jupyter Notebook contains the process for training a Keras network for Optical Character Recognition (OCR) for use with a document scanner app, on the Chars74K Dataset. See the following link for the dataset:**

http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/

In [14]:
# Imports
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from numpy.random import default_rng
import os
import shutil
import math

## Organize Data:

For now, training will only be done on the "Fnt" folder, which contains computer font characters, of 4 different variations. This data comes as a single folder with a directory for each class - thus we must split into train/test file sets before using a validation split on the training data generator.

Inspiration from how to handle this in part from here: 
https://stackoverflow.com/questions/46717742/split-data-directory-into-training-and-test-directory-with-sub-directory-structu

and 

https://stackoverflow.com/questions/8505651/non-repetitive-random-number-in-numpy

In [17]:
trainDataLocation = '/Users/danielmallia/Documents/TTP/IndependentStudy/Capstone/Data/English/Fnt/'
testDataLocation = '/Users/danielmallia/Documents/TTP/IndependentStudy/Capstone/Data/English/FntTest/'

# For each sample folder
for folder in os.listdir(trainDataLocation):
    if(folder == '.DS_Store'): # Ignore .DS_Store files
        continue
    else: 
        # Make a matching test folder
        testPath = testDataLocation + folder + '/'
        os.mkdir(testPath)
        
        currentFolderPath = trainDataLocation + folder + '/'
        FILE_NAME_LIST = os.listdir(currentFolderPath)
        numberOfFiles = len(FILE_NAME_LIST)
        numRange = np.arange(1, numberOfFiles + 1)
        numOfSelections = math.floor(numberOfFiles * .2)
        
        # Randomly select indices for choosing approximately 20% of the samples
        randomNumGen = default_rng()
        testSelections = randomNumGen.choice(numRange, size=numOfSelections, replace=False)
        
        # For each index selected
        for index in testSelections:
            filename = FILE_NAME_LIST[index]
            shutil.move(currentFolderPath + filename, testPath + filename)

62


## Data Preprocessing:

In [ ]:
# Initialize generators
trainDataGen = ImageDataGenerator(rescale=1./255, validation_split=.2)
testDataGen = ImageDataGenerator(rescale=1./255)

# Flow from directories



## Model:

In [ ]:
model = models.Sequential()